<a href="https://colab.research.google.com/github/ananyaarya02/Audio_Summarizer/blob/main/Audio_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install openai-whisper pydub transformers gtts


In [15]:
from transformers import pipeline
import whisper
from pydub import AudioSegment
from transformers import T5Tokenizer, T5ForConditionalGeneration




In [16]:
model = whisper.load_model("base")
# audio to WAV format
def preprocess_audio(file_path):
    audio = AudioSegment.from_file(file_path)
    audio.export("converted_audio.wav", format="wav")
# audio to text
def transcribe_audio(file_path):
    preprocess_audio(file_path)
    result = model.transcribe("converted_audio.wav")
    return result["text"]

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [31]:
t5_model_name = "t5-small"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)

def summarize_text(text):
    inputs = t5_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = t5_model.generate(
        inputs, max_length=300, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
    )
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [19]:
def text_to_audio(text):
    output_audio_path="summary_audio.mp3"
    tts = gTTS(text)
    tts.save(output_audio_path)
    print(f"Summary audio saved as: {output_audio_path}")

In [20]:
def audio_summarizer_pipeline(audio_file):
    #Convert speech to text
    transcript = transcribe_audio(audio_file)
    #summarize
    summary = summarize_text(transcript)
    #audio summary
    text_to_audio(summary)
    return transcript, summary



Testing


In [11]:
from gtts import gTTS

text = "Artificial Intelligence (AI) is a transformative technology that has revolutionized countless aspects of human life, reshaping industries, enhancing productivity, and driving innovation. At its core, AI refers to the simulation of human intelligence processes by machines, especially computer systems, to perform tasks that typically require human cognition. These processes include learning, reasoning, problem-solving, perception, and language understanding. Machine learning, a subset of AI, enables systems to learn from data and improve their performance over time without explicit programming. Deep learning, a further subfield, uses neural networks to analyze vast amounts of data, achieving breakthroughs in image recognition, natural language processing, and autonomous systems. AI is the backbone of many modern technologies, from virtual assistants like Siri and Alexa to recommendation systems on platforms like Netflix and Amazon. In healthcare, AI assists in early disease detection, personalized medicine, and surgical robotics. In transportation, it powers autonomous vehicles and traffic management systems. AI's applications extend to finance, education, agriculture, and even entertainment, where it creates immersive experiences and generates realistic content. Despite its immense potential, AI also raises ethical concerns, such as biases in algorithms, privacy issues, and the displacement of jobs. As AI continues to evolve, its development must be guided by principles of fairness, transparency, and accountability to ensure it benefits humanity. With advancements in generative AI, robotics, and edge computing, the future of AI holds promise for solving some of the world's most complex challenges, from climate change to global pandemics, making it one of the most significant technological advancements of the 21st century."
tts = gTTS(text)
audio_file = "test_audio.mp3"
tts.save(audio_file)
print("Test audio file created:", audio_file)


Test audio file created: test_audio.mp3


In [32]:
transcript,summary = audio_summarizer_pipeline(audio_file)
print("transcript:",transcript)
print("summary:",summary)

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Summary audio saved as: summary_audio.mp3
transcript:  Artificial Intelligence AI is a transformative technology that has revolutionized countless aspects of human life. Re-shaping industries, enhancing productivity and driving innovation at its core, AI refers to the simulation of human intelligence processes by machines. Especially computer systems. To perform tasks that typically require human cognition, these processes include learning, reasoning, problem solving, perception, and language understanding, machine learning, a subset of AI, enables systems to learn from data and improve their performance over time without explicit. Programming, deep learning, a further subfield uses neural networks to analyze vast amounts of data, achieving breakthroughs in image recognition, natural language processing, and autonomous systems AI is the backbone of many modern technologies. From virtual assistants like Siri and Alexa to recommendation systems on platforms like Netflix and Amazon. In he